In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
import math
from neuralprophet import NeuralProphet

ModuleNotFoundError: No module named 'neuralprophet'

In [ ]:
file = 'df4eda.csv'
df4eda = pd.read_csv(file, index_col=0)
df4eda

In [ ]:
df = df4eda.copy()
#df.drop(['Zip', 'Region', 'Quarter', 'MonthName', 'Segment', 'RunningMonths', 'isVanArsdel',
#                   'MfgisVanArsdel', 'District', 'MonthIndex', 'MonthID', 'Month',
#                    'IsCompeteHide', 'ManufacturerID_y', 'Rolling Period Sort', 'Running Year',
#                   'Manufacturer', 'Product', 'Rolling Period', 'Running Months', 'ProductID',
#        'Units', 'Category', 'ManufacturerID_x', 'IsCompete', 'MonthNo', 'Year', 'City', 'State'], axis=1, inplace=True)
df = df[['Date','Revenue_T']]
df = df.rename(columns={"Date": "ds"})

In [ ]:
df.info()

In [ ]:
# удаление дубликатов
df = df.drop_duplicates()

#агрегация и суммирование по датам
agg_func_math = {
    'Revenue_T': ['sum']
}

# группирую по дате (месяц-год) и считаю статистические величины и все округляю до второго знака
df = df.groupby(['ds']).agg(agg_func_math).round(2).reset_index()

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print(duplicate_rows_df)
print("number of duplicate rows: ", duplicate_rows_df.shape)

In [ ]:
# в новом датасете создам из названий мультииндексных родительских, объединяя через нижнее подчеркивание
df.columns = ['_'.join(col) for col in df.columns.values]

# сброшу индексы
df = df.reset_index()
df

In [ ]:
df = df.rename(columns={"ds_": "ds", "Revenue_T_sum": "y"})
df.drop(['index'], axis=1, inplace=True)
df

In [ ]:
#df['ds'] = df4eda['Date']
#df['x'] = df4eda['Revenue_T']
#df = df['ds'].concat(df['x'])
#df

In [ ]:
m = NeuralProphet()

metrics = m.fit(df, freq="D")

forecast = m.predict(df)

In [ ]:
from prophet import Prophet

m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)